In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow
import cv2
import keras
import keras.backend as K
from sklearn.svm import SVC
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from keras import layers
from keras.models import Model,load_model
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras import optimizers
from tensorflow.keras import callbacks
from sklearn.utils.class_weight import compute_class_weight
from keras import regularizers,initializers
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc, roc_auc_score, cohen_kappa_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score

In [ ]:
df=pd.read_csv('../input/histopathologic-cancer-detection/train_labels.csv')
df=df.head(160000)
df=shuffle(df)
input_Examples=10000 #traindf.shape[0]
inputExamples=50
print("No.of training images: ",df.shape[0],"Columns: ",df.shape[1])
print(df.columns)
print(df.head(20))
filenames,labels=df['id'],df['label']

In [ ]:
XTRN = np.zeros(shape=(input_Examples,96,96,3))
for i in range(input_Examples):
    fname= r'../input/histopathologic-cancer-detection/train/'+ filenames[i] + '.tif'
    img = cv2.imread(fname)
    img = cv2.resize(img, (96,96), interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    XTRN[i] = img

height=XTRN.shape[1]
width=XTRN.shape[2]
depth=XTRN.shape[3]
input_shape = (height, width, depth)
chanDim = -1
# if we are using "channels first", update the input shape and channels dimension
if K.image_data_format() == "channels_first":
    input_shape = (depth, height, width)
    chanDim = 1
print("Total Train Images:",XTRN.shape[0],"Width:",width,"Height",height,"Channels:",depth)

In [ ]:
print ("number of training examples = " + str(XTRN.shape[0]))
print ("XTRN shape: " + str(XTRN.shape))
print ("YTRN shape: " + str(YTRN.shape))

In [ ]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(XTRN,YTRN, test_size = 0.10,shuffle=True, random_state = 7)


In [ ]:
lbl_present=np.unique(labels)
numclasses=len(lbl_present)
classes=dict(labels.value_counts())
class_df=pd.DataFrame({"classname":list(classes.keys()),"No.of examples":list(classes.values())})

class_df.plot(kind='bar', label='Class Name', color=[[ 'purple','LIMEGREEN']])
plt.title('Number of images for different label type')
plt.xlabel('label')
plt.ylabel('Counts')
plt.grid(axis='y')

# Data Generator

In [ ]:
def getDataGen(model_name,input_shape):
    height,width,channels = input_shape
    preprocess_func = None
    def zscoreNorm(x):
        return (x - x.mean()) / x.std()  if x.std() > 0 else x
    def minmax(x):
        return x/255.0
        
    if model_name=='vgg16':
        preprocess_func = minmax
    if model_name=='pretrain_vgg16':
        preprocess_func = keras.applications.vgg16.preprocess_input
    if model_name=='pretrain_ResNet50':
        preprocess_func = keras.applications.resnet.preprocess_input
    if model_name=='pretrain_InceptionV3':
        preprocess_func = keras.applications.inception_v3.preprocess_input
    if model_name=='pretrain_DenseNet121':
        preprocess_func = keras.applications.densenet.preprocess_input
    if model_name=='pretrain_MobileNet':
        preprocess_func = keras.applications.mobilenet.preprocess_input
    if model_name=='HCD-CNN':
        preprocess_func = minmax

    datagenTrain = ImageDataGenerator(
                        featurewise_center=False,
                        samplewise_center=False,
                        featurewise_std_normalization=False,
                        samplewise_std_normalization=False,
                        #rotation_range=90,
                        #width_shift_range=0.2,#[-200,200] 
                        #height_shift_range=0.2, 
                        #brightness_range = [0.5,1.0],
                        #shear_range=0.1,
                        #zoom_range = [0.8,0.8], # Lower value more zoom!!
                        horizontal_flip=True,  
                        vertical_flip=True,
                        #fill_mode='nearest',
                        preprocessing_function=preprocess_func
                )
    datagenTest = ImageDataGenerator(
                        #horizontal_flip=True,  
                        #vertical_flip=True,
                        #rescale=1.0/255
                        preprocessing_function=preprocess_func
                )
    print(preprocess_func)
    return datagenTrain, datagenTest

# Train,Test,val split  // using flow_from_directory()

In [ ]:
try:
    os.mkdir('img_dir')
except FileExistsError:
    pass
img_dir=os.path.join('../input/output/','img_dir')
dfData=shuffle(pd.concat([filenames,labels],axis=1).reset_index(drop=True))
y=dfData['label']
dfTrain,dfTest = train_test_split(dfData, test_size = 0.10,shuffle=True, random_state = 7,stratify=y)
y=dfTrain['label']
df_train,df_val = train_test_split(dfTrain, test_size = 0.10,shuffle=True, random_state = 7,stratify=y)
trainPath='img_dir/train'
valPath='img_dir/val'
testPath='img_dir/test'
for root_dir in [trainPath,valPath,testPath]:
    for subf in ['0','1']:
        try:
            os.makedirs(os.path.join(root_dir, subf))
        except FileExistsError:
            pass
classCountsTrain=pd.DataFrame(np.array(np.unique(df_train['label'], return_counts=True)).T, columns=['Class','Freq']).set_index('Class')
classCountsTrain['Freq'] = classCountsTrain['Freq'] / np.sum(classCountsTrain['Freq'])

#print(classCountsTrain)
classCountsTrain.plot(kind='bar',label='Class',color=[['purple','LIMEGREEN']])
plt.title('Class Proportion in Training:')
plt.xlabel('label')
plt.ylabel('Counts')
plt.grid(axis='y')

classCountsVal=pd.DataFrame(np.array(np.unique(df_val['label'], return_counts=True)).T, columns=['Class','Freq']).set_index('Class')
classCountsVal['Freq'] = classCountsVal['Freq'] / np.sum(classCountsVal['Freq'])
#print(classCountsVal)
classCountsVal.plot(kind='bar', label='Class', color=[[ 'purple', 'LIMEGREEN']])
plt.title('Class Proportion in Validation:')
plt.xlabel('label')
plt.ylabel('Counts')
plt.grid(axis='y')
dfData
weight = compute_class_weight('balanced', np.unique(df_train['label']), df_train['label'])
weights = {i : weight[i] for i in range(2)}

In [ ]:
src_path='../input/histopathologic-cancer-detection/train'

for i in range(len(df_train)):
    fname=df_train.iloc[i,0] + '.tif'
    label=df_train.iloc[i,1]
    src = os.path.join(src_path, fname)
    dst = os.path.join(trainPath, str(label), fname)
    shutil.copyfile(src, dst)

for i in range(len(df_val)):
    fname=df_val.iloc[i,0] + '.tif'
    label=df_val.iloc[i,1]
    src = os.path.join(src_path, fname)
    dst = os.path.join(valPath, str(label), fname)
    shutil.copyfile(src, dst)

for i in range(len(dfTest)):
    fname=dfTest.iloc[i,0] + '.tif'
    label=dfTest.iloc[i,1]
    src = os.path.join(src_path, fname)
    dst = os.path.join(testPath,str(label), fname)
    shutil.copyfile(src, dst)

In [ ]:
height=96
width=96
depth=3
input_shape = (height, width, depth)
chanDim = -1
# if we are using "channels first", update the input shape and channels dimension
if K.image_data_format() == "channels_first":
    input_shape = (depth, height, width)
    chanDim = 1
num_train_samples = len(df_train)
num_val_samples = len(df_val)
num_test_samples = len(dfTest)
print("Total Train Images:",num_train_samples,"Width:",width,"Height",height,"Channels:",depth)
print("Total Validation Images:",num_val_samples,"Width:",width,"Height",height,"Channels:",depth)
print("Total Test Images:",num_test_samples,"Width:",width,"Height",height,"Channels:",depth)

# Training

In [ ]:
def getOptimizer(optimizer_name='Adam',lr=0.01):
    if optimizer_name=='Adam':
        optimizer = optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-12, decay=1e-6, amsgrad=False)
    elif optimizer_name=='SGD':
        optimizer=optimizers.SGD(lr=lr, decay=1e-6, momentum=0.0, nesterov=False)
    elif optimizer_name=='SGD_Momentum':
        optimizer=optimizers.SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=False)
    elif optimizer_name=='SGD_Nesterov':
        optimizer=optimizers.SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    elif optimizer_name=='RMSprop':
        optimizer=optimizers.RMSprop(lr=lr, rho=0.9, epsilon=1e-12, decay=0.0)
    elif optimizer_name=='Adagrad':
        optimizer=optimizers.Adagrad(lr=lr, epsilon=None, decay=0.0)
    elif optimizer_name=='Adadelta':
        optimizer=optimizers.Adadelta(lr=lr, rho=0.95, epsilon=None, decay=0.0)
    elif optimizer_name=='Adamax':
        optimizer=optimizers.Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    elif optimizer_name=='Nadam':
        optimizer=optimizers.Nadam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004) #Nesterov Adam optimizer
    return optimizer

In [ ]:
learning_rate_reduction = callbacks.ReduceLROnPlateau(  monitor='val_loss', 
                                                                    patience=10, 
                                                                    verbose=1, 
                                                                    factor=0.5,
                                                                    min_lr=0.00001,
                                                                    min_delta=1e-4, 
                                                                    mode='min')

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', 
                                         min_delta=1e-6, 
                                         patience=100, 
                                         verbose=1, 
                                         mode='min')

In [ ]:
batch_size=32
epochs = 50

In [ ]:
def plot_History(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].grid(True, axis='both')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['Train','Validation'], loc='upper left')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].grid(True, axis='both')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['Train','Validation'], loc='upper left')
    plt.show()

In [ ]:
def train_model(model_name,model,optimizer,numclasses,X_Train,Y_Train,chanDim=-1,batch_size=32,epochs=10):
    class_counts=pd.DataFrame(np.array(np.unique(Y_Train,return_counts=True)).T,columns=['class','frequency'])
    class_counts['frequency']=class_counts['frequency']/np.sum(class_counts['frequency'])
    print("Class Proportion in Train Set")
    print(class_counts)
    checkpoint_saving = callbacks.ModelCheckpoint(filepath="../input/output/weight_dir/chkPnt"+model_name+".h5",
                                              save_weights_only=True,
                                              monitor='val_acc', 
                                              save_best_only=True,  
                                              mode='max')
    if model_name='pretrain_InceptionV3':
        X_Train=keras.applications.inception_v3.preprocess_input(X_Train)
    if model_name='pretrain_MobileNet': 
        X_Train=keras.applications.mobilenet.preprocess_input(X_Train)
        
    X_train, X_val, Y_train, Y_val = train_test_split(X_Train,Y_Train, test_size = 0.10,shuffle=True, random_state = 7)
    print("Train and validation size :",X_train.shape,X_val.shape,Y_train.shape,Y_val.shape)
    class_counts=pd.DataFrame(np.array(np.unique(Y_train,return_counts=True)).T,columns=['class','frequency'])
    class_counts['frequency']=class_counts['frequency']/np.sum(class_counts['frequency'])
    print("Class Proportion in Training:")
    print(class_counts)
    Y_train=to_categorical(Y_train,num_classes=numclasses,dtype='float64')
    Y_val=to_categorical(Y_val,num_classes=numclasses,dtype='float64')
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
   

    datagenTrain, datagenTest = getDataGen(model_name,input_shape)
    augmented_train_data=datagenTrain.flow(X_train,Y_train,batch_size=batch_size,
                                     shuffle=True)
    augmented_val_data=datagenTest.flow(X_val,Y_val,batch_size=batch_size,
                                     shuffle=False)
                                    
    history_Training=model.fit_generator(augmented_train_data,
                                         steps_per_epoch=np.ceil(X_train.shape[0]//batch_size),
                                         epochs=epochs,
                                         verbose=1,
                                         validation_data=augmented_val_data,
                                         validation_steps=np.ceil(X_val.shape[0]//batch_size),
                                         class_weight=weight,
                                         callbacks=[checkpoint_saving,learning_rate_reduction])
    plot_History(history_Training)
    print('------------------------------------------------------------------------')
    print('Mean Validation Score: Loss of',np.mean(history_Training.history['val_loss']), 'Mean Accuracy of:', np.mean(history_Training.history['val_accuracy'])*100)
    print('------------------------------------------------------------------------')
    return model

In [ ]:
def train_model_ffd(model_name,model,optimizer_name,lr,numclasses,chanDim=-1,batch_size=32,epochs=10,compile_model=True):
    weights_Path=model_name+'.h5'
    heigh,width,channel=input_shape
    datagenTrain, datagenTest = getDataGen(model_name,input_shape)
    
    train_gen = datagenTrain.flow_from_directory(trainPath,
                                                target_size=(height,width),
                                                batch_size=batch_size,
                                                shuffle=True,
                                                interpolation='lanczos',
                                                class_mode='categorical')
    val_gen = datagenTest.flow_from_directory(valPath,
                                                target_size=(height,width),
                                                batch_size=batch_size,
                                                shuffle=True,
                                                interpolation='lanczos',
                                                class_mode='categorical')
   
        
    optimizer= getOptimizer(optimizer_name,lr)
    if compile_model==True:
        model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
   
    checkpoint_saving = callbacks.ModelCheckpoint(filepath="chkPnt"+model_name+".h5",
                                              save_weights_only=True,
                                              monitor='val_accuracy', 
                                              save_best_only=True,  
                                              mode='max')
   
    history_Training=model.fit_generator(train_gen,
                                         steps_per_epoch=np.ceil(num_train_samples/batch_size),
                                         epochs=epochs,
                                         verbose=1,
                                         validation_data=val_gen,
                                         validation_steps=np.ceil(num_val_samples/batch_size),
                                         class_weight=weights,
                                         callbacks=[early_stopping,checkpoint_saving,learning_rate_reduction])
    model.save(weights_Path,include_optimizer=True)
    plot_History(history_Training)
    print('available metrics',model.metrics_names)
    print('------------------------------------------------------------------------')
    print('Mean Validation Score: Loss of',np.mean(history_Training.history['val_loss']), 'Mean Accuracy of:', np.mean(history_Training.history['val_accuracy'])*100)
    print('------------------------------------------------------------------------')

    return model
    

# Grid search

In [ ]:
optimizer_name=['Adam','SGD_Momentum','SGD_Nesterov']
lr=np.power(10,np.arange(-3,-1,dtype=float))
neuron=[512,1024]
dropRate=[0.1,0.2,0.3,0.4,0.5]


In [ ]:
def gridSearch_ffd(model_name,numclasses,paramGrid,chanDim=-1,batch_size=32,epochs=10):
    if paramGrid!=None and len(paramGrid)>0:
        heigh,width,channel=input_shape
        datagenTrain, datagenTest = getDataGen(model_name,input_shape)
        train_gen = datagenTrain.flow_from_directory(trainPath,
                                                        target_size=(height,width),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        interpolation='lanczos',
                                                        class_mode='categorical')
        val_gen = datagenTest.flow_from_directory(valPath,
                                                    target_size=(height,width),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    interpolation='lanczos',
                                                    class_mode='categorical')
        best_param={'model_name':model_name,'optimizer':None,'lr':0,'neuron':0,'dropRate':0,'acc_score':0}
        for optimizer_name,lr,neuron,dropRate in paramGrid:
            optimizer= getOptimizer(optimizer_name,lr)
            if model_name=='vgg16': 
                model=vgg16scratch(numclasses,input_shape,chanDim,'he_normal',neuron,dropRate)
            if model_name=='pretrain_vgg16': 
                model=vgg16_pretrained(numclasses,input_shape,chanDim,'he_normal',neuron,dropRate)
            if model_name=='pretrain_InceptionV3':
                model=inceptionv3_pretrained(numclasses,input_shape,chanDim,'he_normal',neuron,dropRate)
            if model_name=='pretrain_MobileNet': 
                model=mobilenet_pretrained(numclasses,input_shape,chanDim,'he_normal',neuron,dropRate)
            if model_name=='pretrain_ResNet50':
                model=resnet50_pretrained(numclasses,input_shape,chanDim,'he_normal',neuron,dropRate)
            if model_name=='pretrain_DenseNet121':
                model=densenet121_pretrained(numclasses,input_shape,chanDim,'he_normal',neuron,dropRate)
            if model_name=='HCD-CNN':
                model=testNet(numclasses,input_shape,chanDim,'he_normal',neuron,dropRate)

            model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

            history_Training=model.fit_generator(train_gen,
                                                 steps_per_epoch=np.ceil(num_train_samples/batch_size),
                                                 epochs=epochs,
                                                 verbose=0,
                                                 validation_data=val_gen,
                                                 validation_steps=np.ceil(num_val_samples/batch_size),
                                                 class_weight=weights)   
            val_acc=np.mean(history_Training.history['val_accuracy'])*100
            val_loss=np.mean(history_Training.history['val_loss'])
            if val_acc>best_param['acc_score']:
                best_param['optimizer']=optimizer_name
                best_param['lr']=lr
                best_param['neuron']=neuron
                best_param['dropRate']=dropRate
                best_param['acc_score']=val_acc
            print('hyperparameters for the model',model_name,'with optimizer =',optimizer_name,'learning rate=',lr,'number of neurons=',neuron,'dropout rate =', dropRate,'achieved mean validation accuracy of',val_acc,' achieved mean validation loss of',val_loss)
            try:
                del model
            except NameError:
                pass
        print('Best hyperparameters for the model',model_name,'with optimizer =',best_param['optimizer'],'learning rate=',best_param['lr'],'number of neurons=',best_param['neuron'],'dropout rate =', best_param['dropRate'],'achieved validation accuracy of',best_param['acc_score'])
            

    return best_param.values()
    

# MODELS
# VGG16

In [ ]:
def vgg16scratch(numclasses,input_shape=None,channel_dim=-1,initializer='he_normal',neuron=1024,dropRate=0.2):
    image_input=layers.Input(shape=input_shape)
    #Block 1:
    x=layers.Conv2D(64,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block1_Conv1')(image_input)
    x=layers.Conv2D(64,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block1_Conv2')(x)
    x=layers.MaxPooling2D((2,2),strides=(2,2),name='Block1_Pool')(x)
    #Block 2:
    x=layers.Conv2D(128,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block2_Conv1')(x)
    x=layers.Conv2D(128,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block2_Conv2')(x)
    x=layers.MaxPooling2D((2,2),strides=(2,2),name='Block2_Pool')(x)
    #Block 3:
    x=layers.Conv2D(256,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block3_Conv1')(x)
    x=layers.Conv2D(256,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block3_Conv2')(x)
    x=layers.Conv2D(256,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block3_Conv3')(x)
    x=layers.MaxPooling2D((2,2),strides=(2,2),name='Block3_Pool')(x)
    #Block 4:
    x=layers.Conv2D(512,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block4_Conv1')(x)
    x=layers.Conv2D(512,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block4_Conv2')(x)
    x=layers.Conv2D(512,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block4_Conv3')(x)
    x=layers.MaxPooling2D((2,2),strides=(2,2),name='Block4_Pool')(x)
    #Block 5:
    x=layers.Conv2D(512,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block5_Conv2')(x)
    x=layers.Conv2D(512,(3,3),strides=(1,1),activation='relu',padding='same',kernel_initializer=initializer,name='Block5_Conv3')(x)
    x=layers.MaxPooling2D((2,2),strides=(2,2),name='Block5_Pool')(x)
   # x=layers.Flatten(name='flatten')(x)
    x=layers.GlobalAveragePooling2D()(x)
    x=layers.Dense(neuron,activation='relu',name='FC1',kernel_initializer=initializer,bias_initializer=initializer)(x)
    x=layers.Dense(neuron,activation='relu',name='FC2',kernel_initializer=initializer,bias_initializer=initializer)(x)
    x=layers.Dense(numclasses,activation='softmax',name='prediction')(x)
    model=Model(image_input,x,name='vgg16')
    print(model.summary())
    return model

In [ ]:
paramGrid=[('SGD_Momentum',0.01,dl,0.2) for dl in neuron ]
paramGrid

In [ ]:
optimizer_name,lr,neuron,dropRate,acc=gridSearch_ffd('vgg16',numclasses,paramGrid,chanDim=-1,batch_size=batch_size,epochs=epochs)


In [ ]:
vgg16_Scratch=vgg16scratch(numclasses,input_shape,-1,'he_normal',512,0.2)

In [ ]:
modelVGG16Scratch=train_model('vgg16',vgg16_Scratch,optimizer,numclasses,X_Train, Y_Train,chanDim=-1,batch_size=batch_size,epochs=epochs)

In [ ]:
train_VGG16=train_model_ffd('vgg16',vgg16_Scratch,'SGD_Momentum',0.01,numclasses,chanDim=-1,batch_size=batch_size,epochs=epochs)

In [ ]:
def vgg16_pretrained(numclasses,input_shape,chanDim=-1,initializer='he_normal',neuron=1024,dropRate=0.2):
    base_model=keras.applications.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=input_shape,
                                        pooling='avg')
     #base_model.load_weights(pathtoh5)
    #for i,layer in enumerate(base_model.layers):
        #print(i,layer.name)
    for layer in base_model.layers[:11]:
        layer.trainable=False
    for layer in base_model.layers[11:]:
        layer.trainable=True
    x=base_model.output
    x=layers.Dense(neuron,activation='relu',name='FC1',kernel_initializer=initializer,bias_initializer=initializer,use_bias=True,kernel_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(x)
    x=layers.Dropout(rate=dropRate,name='Dropout_1')(x)
    
    x=layers.BatchNormalization(axis=chanDim,name='BatchNorm_1',beta_initializer=initializer,gamma_initializer=initializer,beta_regularizer=regularizers.l2(0.01),gamma_regularizer=regularizers.l2(0.01))(x)    
    x=layers.Dense(neuron,activation='relu',name='FC2',kernel_initializer=initializer,bias_initializer=initializer,use_bias=True,kernel_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(x)
    x=layers.Dropout(rate=dropRate,name='Dropout_2')(x)
    
    x=layers.BatchNormalization(axis=chanDim,name='BatchNorm_2',beta_initializer=initializer,gamma_initializer=initializer,beta_regularizer=regularizers.l2(0.01),gamma_regularizer=regularizers.l2(0.01))(x)  
    x=layers.Activation('relu',name='Act_Relu_1')(x)
    x=layers.Dense(numclasses,activation='softmax',name='Classification')(x)
    model=Model(inputs=base_model.input,outputs=x,name='pretrained_vgg16')
    print(model.summary())
    return model

In [ ]:
paramGrid=[('SGD_Momentum',0.01,dl,dr) for dl in neuron for dr in dropRate]
paramGrid

In [ ]:
optimizer_name,lr,neuron,dropRate,acc=gridSearch_ffd('pretrain_vgg16',numclasses,paramGrid,chanDim=-1,batch_size=batch_size,epochs=epochs)


In [ ]:
modelVGG16Pretrained=vgg16_pretrained(numclasses,input_shape,-1,'he_normal',512,0.1)

In [ ]:
train_VGG16Pretrained=train_model('pretrained_vgg16',modelVGG16Pretrained,optimizer,numclasses,X_Train, Y_Train,chanDim=-1,batch_size=batch_size,epochs=epochs)

In [ ]:
train_VGG16Pretrained=train_model_ffd('pretrain_vgg16',modelVGG16Pretrained,'SGD_Momentum',0.01,numclasses,chanDim=-1,batch_size=batch_size,epochs=epochs)

# ResNet50

In [ ]:
def resnet50_pretrained(numclasses,input_shape,chanDim=-1,initializer='he_normal',neuron=1024,dropRate=0.2):
    base_model=keras.applications.ResNet50(include_top=False,
                                        weights='imagenet',
                                        input_shape=input_shape,
                                        pooling='avg')
   # for i,layer in enumerate(base_model.layers):
        #print(i,layer.name)
    for layer in base_model.layers[:154]:
        layer.trainable=False
    for layer in base_model.layers[154:]:
        layer.trainable=True
    x=base_model.output
    x=layers.Dense(neuron,activation='relu',name='FC1',kernel_initializer=initializer,bias_initializer=initializer,use_bias=True,kernel_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(x)
    x=layers.BatchNormalization(axis=chanDim,name='BatchNorm_final',beta_initializer=initializer,gamma_initializer=initializer,beta_regularizer=regularizers.l2(0.01),gamma_regularizer=regularizers.l2(0.01))(x)  
    x=layers.Activation('relu',name='Act_ReLu_final')(x)
    x=layers.Dropout(rate=dropRate,name='Dropout_final')(x)
    x=layers.Dense(numclasses,activation='softmax',name='Classification')(x)
    model=Model(inputs=base_model.input,outputs=x,name='pretrain_ResNet50')
    print(model.summary())
    return model

In [ ]:
paramGrid=[('SGD_Nesterov',0.01,dl,dr) for dl in neuron for dr in dropRate]
paramGrid

In [ ]:
model,optimizer_name,lr,neuron,dropRate,acc_score=gridSearch_ffd('pretrain_ResNet50',numclasses,paramGrid,chanDim=-1,batch_size=batch_size,epochs=epochs)


In [ ]:
modelresnet50Pretrained=resnet50_pretrained(numclasses,input_shape,-1,'he_normal',512,0.5)

In [ ]:
train_resnet50Pretrained=train_model('pretrain_ResNet50',modelresnet50Pretrained,optimizer,numclasses,X_Train, Y_Train,chanDim=-1,batch_size=batch_size,epochs=epochs)

In [ ]:
train_resnet50Pretrained=train_model_ffd('pretrain_ResNet50',modelresnet50Pretrained,'SGD_Nesterov',0.01,numclasses,chanDim=-1,batch_size=batch_size,epochs=epochs)

# InceptionV3

In [ ]:
def inceptionv3_pretrained(numclasses,input_shape,chanDim=-1,initializer='he_normal',neuron=1024,dropRate=0.2):
    base_model=keras.applications.InceptionV3(include_top=False,
                                        weights='imagenet',
                                        input_shape=input_shape,
                                        pooling='avg')
   # for i,layer in enumerate(base_model.layers):
       # print(i,layer.name)
    for layer in base_model.layers[:249]:
        layer.trainable=False
    for layer in base_model.layers[249:]:
        layer.trainable=True
    x=base_model.output
    x=layers.Dense(neuron,activation='relu',name='FC1',kernel_initializer=initializer,bias_initializer=initializer,use_bias=True,kernel_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(x)
    x=layers.BatchNormalization(axis=chanDim,name='BatchNorm_final',beta_initializer=initializer,gamma_initializer=initializer,beta_regularizer=regularizers.l2(0.01),gamma_regularizer=regularizers.l2(0.01))(x)  
    x=layers.Activation('relu',name='Act_ReLu_final')(x)
    x=layers.Dropout(rate=dropRate,name='Dropout_final')(x)
    x=layers.Dense(numclasses,activation='softmax',name='Classification')(x)
    model=Model(inputs=base_model.input,outputs=x,name='pretrain_InceptionV3')
   # print(model.summary())
    return model

In [ ]:
paramGrid=[('RMSprop',0.01,dl,dr) for dl in neuron for dr in dropRate]
paramGrid

In [ ]:
model,optimizer_name,lr,neuron,dropRate,acc_score=gridSearch_ffd('pretrain_InceptionV3',numclasses,paramGrid,chanDim=-1,batch_size=batch_size,epochs=epochs)


In [ ]:
modelinceptionv3Pretrained=inceptionv3_pretrained(numclasses,input_shape,-1,'he_normal',512,0.1)

In [ ]:
train_inceptionv3Pretrained=train_model('pretrain_InceptionV3',modelinceptionv3Pretrained,optimizer,numclasses,X_Train, Y_Train,chanDim=-1,batch_size=batch_size,epochs=epochs)

In [ ]:
train_inceptionv3Pretrained=train_model_ffd('pretrain_InceptionV3',modelinceptionv3Pretrained,'RMSprop',0.01,numclasses,chanDim=-1,batch_size=batch_size,epochs=epochs)

# DenseNet121

In [ ]:
def densenet121_pretrained(numclasses,input_shape,chanDim=-1,initializer='he_normal',neuron=1024,dropRate=0.2):
    base_model=keras.applications.DenseNet121(include_top=False,
                                        weights='imagenet',
                                        input_shape=input_shape,
                                        pooling='avg')
   # for i,layer in enumerate(base_model.layers):
        #print(i,layer.name)
    
    for layer in base_model.layers[:313]:
        layer.trainable=False
    for layer in base_model.layers[313:]:
        layer.trainable=True
    x=base_model.output
    x=layers.Dense(neuron,activation='relu',name='FC1',kernel_initializer=initializer,bias_initializer=initializer,use_bias=True,kernel_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(x)
    x=layers.BatchNormalization(axis=chanDim,name='BatchNorm_final',beta_initializer=initializer,gamma_initializer=initializer,beta_regularizer=regularizers.l2(0.01),gamma_regularizer=regularizers.l2(0.01))(x)  
    x=layers.Activation('relu',name='Act_ReLu_final')(x)
    x=layers.Dropout(rate=dropRate,name='Dropout_final')(x)
    x=layers.Dense(numclasses,activation='softmax',name='Classification')(x)
    model=Model(inputs=base_model.input,outputs=x,name='pretrain_DenseNet121')
    #print(model.summary())
    return model

In [ ]:
paramGrid=[('SGD_Nesterov',0.1,dl,dr) for dl in neuron for dr in dropRate ]
paramGrid

In [ ]:
model,optimizer_name,lr,neuron,dropRate,acc_score=gridSearch_ffd('pretrain_DenseNet121',numclasses,paramGrid,chanDim=-1,batch_size=batch_size,epochs=epochs)


In [ ]:
modeldensenet121Pretrained=densenet121_pretrained(numclasses,input_shape,-1,'he_normal',512,0.4)

In [ ]:
train_densenet121Pretrained=train_model('pretrain_DenseNet121',modeldensenet121Pretrained,optimizer,numclasses,X_Train, Y_Train,chanDim=-1,batch_size=batch_size,epochs=epochs)

In [ ]:
train_densenet121Pretrained=train_model_ffd('pretrain_DenseNet121',modeldensenet121Pretrained,'SGD_Nesterov',0.1,numclasses,chanDim=-1,batch_size=batch_size,epochs=epochs)

# MobileNet

In [ ]:
def mobilenet_pretrained(numclasses,input_shape,chanDim=-1,initializer='he_normal',neuron=1024,dropRate=0.2):
    base_model=keras.applications.MobileNet(include_top=False,
                                        weights='imagenet',
                                        input_shape=input_shape,
                                        pooling='avg')
    for i,layer in enumerate(base_model.layers):
        print(i,layer.name)
    for layer in base_model.layers[:44]:
        layer.trainable=False
    for layer in base_model.layers[44:]:
        layer.trainable=True
    x=base_model.output
    x=layers.Dense(neuron,activation='relu',name='FC1',kernel_initializer=initializer,bias_initializer=initializer,use_bias=True,kernel_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(x)
    x=layers.BatchNormalization(axis=chanDim,name='BatchNorm_final',beta_initializer=initializer,gamma_initializer=initializer,beta_regularizer=regularizers.l2(0.01),gamma_regularizer=regularizers.l2(0.01))(x)  
    x=layers.Activation('relu',name='Act_ReLu_final')(x)
    x=layers.Dropout(rate=dropRate,name='Dropout_final')(x)
    x=layers.Dense(numclasses,activation='softmax',name='Classification')(x)
    model=Model(inputs=base_model.input,outputs=x,name='pretrain_MobileNet')
    print(model.summary())
    return model

In [ ]:
paramGrid=[('RMSprop',0.001,dl,dr) for dl in neuron for dr in dropRate ]
paramGrid

In [ ]:
model,optimizer_name,lr,neuron,dropRate,acc_score=gridSearch_ffd('pretrain_MobileNet',numclasses,paramGrid,chanDim=-1,batch_size=batch_size,epochs=epochs)


In [ ]:
modelmobilenetPretrained=mobilenet_pretrained(numclasses,input_shape,-1,'he_normal',512,0.2)

In [ ]:
train_mobilenetPretrained=train_model('pretrain_MobileNet',modelmobilenetPretrained,optimizer,numclasses,X_Train, Y_Train,chanDim=-1,batch_size=batch_size,epochs=epochs)

In [ ]:
train_mobilenetPretrained=train_model_ffd('pretrain_MobileNet',modelmobilenetPretrained,'RMSprop',0.001,numclasses,chanDim=-1,batch_size=batch_size,epochs=epochs)

# testNet

In [ ]:
def conv2d_BNRelU(x, filters, kernelSize=(1,1), padding='same', strides=(1, 1),  name=None,chanDim=-1,initializer='he_normal'):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
        act_name =name+'_act'
    else:
        bn_name = None
        conv_name = None
        act_name=None
    x = layers.Conv2D(filters, kernelSize ,strides=strides,padding=padding,use_bias=False,kernel_initializer=initializer, kernel_regularizer=regularizers.l2(0.0002),name=conv_name)(x)
    x = layers.BatchNormalization(axis=chanDim, scale=False, name=bn_name)(x)
    x = layers.Activation('relu', name=act_name)(x)
    return x

In [ ]:
def conv2dR_BNRelU(x, filters, kernelSize=(1,1), padding='same', strides=(1, 1),  name=None,chanDim=-1,initializer='he_normal'):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
        act_name =name+'_act'
    else:
        bn_name = None
        conv_name = None
        act_name=None
    x = layers.BatchNormalization(axis=chanDim, scale=False, name=bn_name)(x)
    x = layers.Activation('relu', name=act_name)(x)
    x = layers.Conv2D(filters, kernelSize ,strides=strides,padding=padding,use_bias=False,kernel_initializer=initializer, kernel_regularizer=regularizers.l2(0.0002),name=conv_name)(x)
    return x

In [ ]:
def testNet(numclasses,input_shape=None,chanDim=-1,initializer='he_normal',neuron=1024,dropRate=0.2):
    img_input = layers.Input(shape=input_shape)
    #block 0
    x=conv2d_BNRelU(img_input,32,(7,7),padding='same',strides=(2,2),name='Bl_0_conv7')
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same',name='Bl_0_pool')(x)
    
    #block 1
    x11=conv2d_BNRelU(x,64,(1,1),padding='valid',strides=(1,1),name='Bl_11_conv1')
    
    x12=conv2d_BNRelU(x,96,(1,1),padding='valid',strides=(1,1),name='Bl_12_conv1')
    x12=conv2d_BNRelU(x12,192,(3,3),padding='same',strides=(1,1),name='Bl_12_conv3')
    
    x13=conv2d_BNRelU(x,32,(1,1),padding='valid',strides=(1,1),name='Bl_13_conv1')
    x13=conv2d_BNRelU(x13,96,(5,5),padding='same',strides=(1,1),name='Bl_13_conv5')
    
    x14=conv2d_BNRelU(x,32,(1,1),padding='valid',strides=(1,1),name='Bl_14_conv1')
    x14=conv2d_BNRelU(x14,96,(7,7),padding='same',strides=(1,1),name='Bl_14_conv7')
    
    x15=layers.MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='Bl_15_pool')(x)
    x15=layers.BatchNormalization(axis=chanDim, epsilon=1.001e-5, momentum=0.9, beta_initializer=initializer,gamma_initializer=initializer, name= 'Bl_15_pool_bn')(x15)
    x15=layers.Activation('relu', name='Bl_15_Pool_act')(x15)
    x15=conv2d_BNRelU(x15,64,(1,1),padding='valid',strides=(1,1),name='Bl_15_conv1')

    
    #concatenation
    x=keras.layers.concatenate([x11,x12,x13,x14,x15],axis=-1)
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='concat_1_pool')(x)
    
    #block 2
    x21=conv2dR_BNRelU(x,128,(1,1),padding='valid',strides=(1,1),name='Bl_2a_conv1')
    x21=conv2dR_BNRelU(x21,32,(3,3),padding='same',strides=(1,1),name='Bl_2a_conv3')
    
    #concatenate 21
    x=keras.layers.concatenate([x,x21],axis=-1)
    
    x22=conv2dR_BNRelU(x,128,(1,1),padding='valid',strides=(1,1),name='Bl_2b_conv1')
    x22=conv2dR_BNRelU(x22,32,(3,3),padding='same',strides=(1,1),name='Bl_2b_conv3')
    
    #concatenate 22
    x=keras.layers.concatenate([x,x22],axis=-1)
    
    x23=conv2dR_BNRelU(x,128,(1,1),padding='valid',strides=(1,1),name='Bl_2c_conv1')
    x23=conv2dR_BNRelU(x22,32,(3,3),padding='same',strides=(1,1),name='Bl_2c_conv3')
    
    #concatenate 23
    x=keras.layers.concatenate([x,x23],axis=-1)
    
    #block 3
    x=layers.BatchNormalization(axis=chanDim, epsilon=1.001e-5, momentum=0.9, beta_initializer=initializer,gamma_initializer=initializer, name= 'Bl_3_bn')(x)
    x=layers.Activation('relu', name='Bl_3_dense_act')(x)
    x=layers.Conv2D(int(int(x.shape[chanDim])*0.5),(1,1) ,strides=(1,1),padding='valid',use_bias=False,kernel_initializer=initializer, kernel_regularizer=regularizers.l2(0.0002),name='Bl_3_conv1')(x)

    x=layers.Dropout(rate=dropRate,name='Dropout_1')(x)
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid',name='Bl_3_pool')(x)
    x=layers.BatchNormalization(axis=chanDim, epsilon=1.001e-5, momentum=0.9, beta_initializer=initializer,gamma_initializer=initializer, name= 'Bl_3_Pool_bn')(x)
    x=layers.Activation('relu', name='Bl_3_pool_act')(x)
    
    #block 4
    x=conv2d_BNRelU(x,512,(1,1),padding='valid',strides=(1,1),name='Bl_4_conv1')
    x=conv2d_BNRelU(x,1024,(3,3),padding='same',strides=(1,1),name='Bl_4_conv3')
    x=conv2d_BNRelU(x,1024,(3,3),padding='same',strides=(1,1),name='Bl_4_conv3_1')
    
    x=layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    
    x=layers.Dense(neuron,activation='relu',name='FC1',kernel_initializer=initializer,bias_initializer=initializer,use_bias=True,kernel_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(x)

    x=layers.BatchNormalization(axis=chanDim, epsilon=1.001e-5, momentum=0.9, beta_initializer=initializer,gamma_initializer=initializer, name= 'Bl_4_dense_bn')(x)
    x=layers.Activation('relu', name='Bl_4_dense_act')(x)
    x=layers.Dropout(rate=dropRate,name='Dropout_2')(x)
    
    x=layers.Dense(numclasses,activation='softmax',name='output',kernel_initializer=initializer,bias_initializer=initializer,use_bias=True,kernel_regularizer=regularizers.l2(0.01),bias_regularizer=regularizers.l2(0.01))(x)

    model=Model(img_input,x,name='HCD-CNN')
    print(model.summary())

    return model
    

In [ ]:
paramGrid=[(o,l,dl,0.2) for o in optimizer_name  for l in lr for dl in neuron  ]
paramGrid

In [ ]:
model,optimizer_name,lr,neuron,dropRate,acc_score=gridSearch_ffd('HCD-CNN',numclasses,paramGrid,chanDim=-1,batch_size=batch_size,epochs=epochs)


In [ ]:
modeltestNet=testNet(numclasses,input_shape,-1,'he_normal',1024,0.2)

In [ ]:
train_testNet=train_model('HCD-CNN',modeltestNet,optimizer,numclasses,X_Train, Y_Train,chanDim=-1,batch_size=batch_size,epochs=epochs)

In [ ]:
train_testNet=train_model_ffd('HCD-CNN',modeltestNet,'SGD_Momentum',0.001,numclasses,chanDim=-1,batch_size=batch_size,epochs=epochs)

# Testing

In [ ]:
def getTestGen(model_name,test_path,input_shape,batch_size=32):
    height,width,channels = input_shape
    preprocess_func = None
    def zscoreNorm(x):
        return (x - x.mean()) / x.std()  if x.std() > 0 else x
    def minmax(x):
        return x/255.0
        
    if model_name=='vgg16':
        preprocess_func = minmax
    if model_name=='pretrain_vgg16':
        preprocess_func = keras.applications.vgg16.preprocess_input
    if model_name=='pretrain_ResNet50':
        preprocess_func = keras.applications.resnet.preprocess_input
    if model_name=='pretrain_InceptionV3':
        preprocess_func = keras.applications.inception_v3.preprocess_input
    if model_name=='pretrain_DenseNet121':
        preprocess_func = keras.applications.densenet.preprocess_input
    if model_name=='pretrain_MobileNet':
        preprocess_func = keras.applications.mobilenet.preprocess_input
    if model_name=='HCD-CNN':
        preprocess_func = minmax

    datagenTest = ImageDataGenerator(preprocessing_function=preprocess_func )
    test_gen = datagenTest.flow_from_directory(testPath,
                                                        target_size=(height,width),
                                                        batch_size=batch_size,
                                                        shuffle=False,
                                                        interpolation='lanczos',
                                                        class_mode='categorical')
   
    return test_gen

In [ ]:
weightsPath='./'
def evaluateModelGenFFD(model_name,input_shape,num_samples,batch_size=256):
    model = load_model(weightsPath+model_name+".h5")
    testGen = getTestGen(model_name,testPath,input_shape,batch_size=32)
    val_loss, val_acc = model.evaluate_generator(testGen,steps=np.ceil(num_samples/batch_size),verbose=1)
    print('Validation on Test Set: Validation Loss =',val_loss,'Validation Accuracy =',val_acc)

In [ ]:
evaluateModelGenFFD('vgg16',input_shape,num_test_samples,batch_size=256)

In [ ]:
evaluateModelGenFFD('pretrain_vgg16',input_shape,num_test_samples,batch_size=256)

In [ ]:
evaluateModelGenFFD('pretrain_ResNet50',input_shape,num_test_samples,batch_size=256)

In [ ]:
evaluateModelGenFFD('pretrain_InceptionV3',input_shape,num_test_samples,batch_size=32)

In [ ]:
evaluateModelGenFFD('pretrain_DenseNet121',input_shape,num_test_samples,batch_size=256)

In [ ]:
evaluateModelGenFFD('pretrain_MobileNet',input_shape,num_test_samples,batch_size=256)

In [ ]:
evaluateModelGenFFD('HCD-CNN',input_shape,num_test_samples,batch_size=256)

## Prediction

In [ ]:
pathToPredictions='./'
def predictGenFFD(model_name,input_shape,num_samples,batch_size=32): 
    model = load_model(weightsPath+model_name+".h5")
    testGen = getTestGen(model_name,testPath,input_shape,batch_size=256)
    Y_Pred_Probs = model.predict_generator(testGen,steps=np.ceil(num_samples/batch_size), verbose=1)
    
    #Converting to Interger Encoding by getting the index of Maximum Probability Column
    Y_pred = np.argmax(Y_Pred_Probs,axis = 1)
    Y_True = testGen.classes
    filenames = testGen.filenames
    labels = testGen.class_indices
    labels = dict((v,k) for k,v in labels.items())
    Y_pred_ClassNames = [labels[k] for k in Y_pred]
    Y_true_ClassNames = [labels[k] for k in Y_True]
    probDF = pd.DataFrame(data=list(Y_Pred_Probs),columns=list(labels.values()))
    results=pd.DataFrame({"Filename":filenames,
                          "True Class Names":Y_true_ClassNames,
                          "Predicted Class Names":Y_pred_ClassNames,
                          "True Class Labels":Y_True,
                          "Predicted Class Labels":Y_pred})
    results = pd.concat([results,probDF],axis=1)
    results.to_csv(f"{pathToPredictions}{model_name}results.csv",index=False)
    del model
    del testGen
    return Y_True, Y_pred, Y_Pred_Probs, labels

In [ ]:
Y_True, Y_Pred, Y_Pred_Probs, labels = predictGenFFD('vgg16',input_shape,num_test_samples,batch_size=256)

In [ ]:
num_test_samples

In [ ]:
Y_True, Y_Pred, Y_Pred_Probs, labels = predictGenFFD('pretrain_vgg16',input_shape,num_test_samples,batch_size=256)

In [ ]:
Y_True, Y_Pred, Y_Pred_Probs, labels = predictGenFFD('pretrain_ResNet50',input_shape,num_test_samples,batch_size=256)

In [ ]:
Y_True, Y_Pred, Y_Pred_Probs, labels = predictGenFFD('pretrain_InceptionV3',input_shape,num_test_samples,batch_size=256)

In [ ]:
Y_True, Y_Pred, Y_Pred_Probs, labels = predictGenFFD('pretrain_DenseNet121',input_shape,num_test_samples,batch_size=256)

In [ ]:
Y_True, Y_Pred, Y_Pred_Probs, labels = predictGenFFD('pretrain_MobileNet',input_shape,num_test_samples,batch_size=256)

In [ ]:
Y_True, Y_Pred, Y_Pred_Probs, labels = predictGenFFD('HCD-CNN',input_shape,num_test_samples,batch_size=256)

## Confusion matrix

In [ ]:
pathToFigures='./'
import itertools
def plot_confusion_matrix(cm, classes, model_name):
    normalize=False
    title='Confusion Matrix for '+model_name
    cmap=plt.cm.Oranges
    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=20)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=12)
    plt.yticks(tick_marks, classes, fontsize=12)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',fontsize=14)
    plt.xlabel('Predicted label',fontsize=14)
    plt.savefig(f'{pathToFigures}confMatrix_{model_name}.jpg',dpi=150,bbox_inches='tight')

In [ ]:
confusion_mtx = confusion_matrix(Y_True, Y_Pred)

plot_confusion_matrix(confusion_mtx, list(labels.values()),'vgg16')

In [ ]:
confusion_mtx = confusion_matrix(Y_True, Y_Pred)

plot_confusion_matrix(confusion_mtx, list(labels.values()),'pretrain_vgg16')

In [ ]:
confusion_mtx = confusion_matrix(Y_True, Y_Pred)

plot_confusion_matrix(confusion_mtx, list(labels.values()),'pretrain_ResNet50')

In [ ]:
confusion_mtx = confusion_matrix(Y_True, Y_Pred)

plot_confusion_matrix(confusion_mtx, list(labels.values()),'pretrain_InceptionV3')

In [ ]:
confusion_mtx = confusion_matrix(Y_True, Y_Pred)

plot_confusion_matrix(confusion_mtx, list(labels.values()),'pretrain_DenseNet121')

In [ ]:
confusion_mtx = confusion_matrix(Y_True, Y_Pred)

plot_confusion_matrix(confusion_mtx, list(labels.values()),'pretrain_MobileNet')

In [ ]:
confusion_mtx = confusion_matrix(Y_True, Y_Pred)

plot_confusion_matrix(confusion_mtx, list(labels.values()),'HCD-CNN')

## Accracy, F1 score, recall, precision

In [ ]:
def getPrecisionRecallFScoreSupport(y_true, y_pred):
    prfsMacro = precision_recall_fscore_support(y_true, y_pred, average='macro')
    prfsMicro = precision_recall_fscore_support(y_true, y_pred, average='micro')
    prfsWeighted = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    print('Macro Precision:',prfsMacro[0],'Recall',prfsMacro[1],'F1-Score',prfsMacro[2],'Support',prfsMacro[3])
    print('Micro Precision:',prfsMicro[0],'Recall',prfsMicro[1],'F1-Score',prfsMicro[2],'Support',prfsMicro[3])
    print('Weighted Precision:',prfsWeighted[0],'Recall',prfsWeighted[1],'F1-Score',prfsWeighted[2],'Support',prfsWeighted[3])

In [ ]:
getPrecisionRecallFScoreSupport(Y_True, Y_Pred)

In [ ]:
print(classification_report(Y_True, Y_Pred,target_names=list(labels.values())))

In [ ]:
print('Accuracy:',accuracy_score(Y_True, Y_Pred))
print('Balanced Accuracy:',balanced_accuracy_score(Y_True, Y_Pred))

## ROC curves

In [ ]:

from itertools import cycle
def roc_curve_multiClassOneVsRestMicro(Y_pred_probs,Y_true,num_classes,labels,model_name):
    classes = list(labels.values())
    #Y_true = t(Y_true, classes=list(np.arange(num_classes)))
    Y_true = to_categorical(Y_true, num_classes = num_classes,dtype='float64')

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    closest_one = dict()
    closest_one_tpr = dict()
    closest_one_fpr = dict()
    for i in range(num_classes):
        fpr[i], tpr[i],_= roc_curve(Y_true[:, i], Y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        closest_one[i] = np.argmax(np.abs(tpr[i]))
        closest_one_tpr[i] = tpr[i][closest_one[i]]
        closest_one_fpr[i] = fpr[i][closest_one[i]]

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(Y_true.ravel(), Y_pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    closest_one["micro"] = np.argmax(np.abs(tpr["micro"]))
    closest_one_tpr["micro"] = tpr["micro"][closest_one["micro"]]
    closest_one_fpr["micro"] = fpr["micro"][closest_one["micro"]]
    if num_classes%2==0:
        rows=int(np.ceil(num_classes/2))+1
    else:
        rows=int(np.ceil(num_classes/2))
    cols=2
    plt.tight_layout()
    lw = 2
    plt.figure(figsize=(6*cols,5*rows))
    plt.subplots_adjust(wspace=0.4, hspace=0.4)
    colors = cycle(['navy', 'darkorange', 'cornflowerblue','brown','purple','green','pink','cyan'])
    for i, color in zip(range(num_classes), colors):
        color = color
        pos=int(str(rows)+str(cols)+str(i+1))
        ax=plt.subplot(pos)
        ax.plot(fpr[i], tpr[i], color=color,
                 lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[i])
        ax.plot(closest_one_fpr[i], closest_one_tpr[i], '.', markersize = 12, fillstyle = 'none', c=color, mew=3)   
        
        ax.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        ax.set_xlim([-0.01, 1.00])
        ax.set_ylim([-0.01, 1.01])
        ax.set_xlabel('False Positive Rate', fontsize=14)
        ax.set_ylabel('True Positive Rate', fontsize=14)
        ax.set_title(f'ROC Curve for Class:{classes[i]}', fontsize=16)
        ax.legend(loc="lower right", fontsize=14)
        ax.tick_params(labelsize=12)
        ax.set_aspect('equal')
    pos=int(str(rows)+str(cols)+str(i+2))
    ax=plt.subplot(pos)
    ax.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC Curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='red', linestyle='--', linewidth=2)
    ax.plot(closest_one_fpr["micro"], closest_one_tpr["micro"], '.', markersize = 12, fillstyle = 'none', c='r', mew=3)
    ax.set_xlim([-0.01, 1.00])
    ax.set_ylim([-0.01, 1.01])
    ax.set_xlabel('False Positive Rate', fontsize=14)
    ax.set_ylabel('True Positive Rate', fontsize=14)
    ax.set_title(f'ROC Curve: Micro-Average', fontsize=16)
    ax.legend(loc="lower right", fontsize=14)
    ax.tick_params(labelsize=12)
    ax.set_aspect('equal')
    plt.savefig(f'{pathToFigures}ROC_OVR_Micro_{model_name}.jpg',dpi=150,bbox_inches='tight')
    plt.show()

In [ ]:
roc_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'vgg16')

In [ ]:
roc_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_vgg16')

In [ ]:
roc_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_ResNet50')

In [ ]:
roc_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_InceptionV3')

In [ ]:
roc_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_DenseNet121')

In [ ]:
roc_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_MobileNet')

In [ ]:
roc_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'HCD-CNN')

In [ ]:
def roc_curve_multiClassOneVsOneMacro(Y_pred_probs,Y_true,num_classes,labels,model_name):
    classes = list(labels.values())
    Y_true = to_categorical(Y_true, num_classes = num_classes,dtype='float64')
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    closest_one = dict()
    closest_one_tpr = dict()
    closest_one_fpr = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], thresholds = roc_curve(Y_true[:, i], Y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        closest_one[i] = np.argmax(np.abs(tpr[i]))
        closest_one_tpr[i] = tpr[i][closest_one[i]]
        closest_one_fpr[i] = fpr[i][closest_one[i]]
        
        
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], thresholds = roc_curve(Y_true.ravel(), Y_pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    closest_one["micro"] = np.argmax(np.abs(tpr["micro"]))
    closest_one_tpr["micro"] = tpr["micro"][closest_one["micro"]]
    closest_one_fpr["micro"] = fpr["micro"][closest_one["micro"]]
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(num_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= num_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(figsize=(10,10))
    lw = 2
    plt.plot(fpr["micro"], tpr["micro"],
             label='Micro-Average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]),
             color='red', linestyle=':', linewidth=4)
    plt.plot(closest_one_fpr["micro"], closest_one_tpr["micro"], '.', markersize = 12, fillstyle = 'none', c='r', mew=3)
    
    plt.plot(fpr["macro"], tpr["macro"],
             label='Macro-Average ROC curve (area = {0:0.2f})'.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue','brown','purple','green','pink','cyan'])
    for i, color in zip(range(num_classes), colors):
        color = color
        plt.plot(fpr[i], tpr[i], color=color, lw=lw, label='ROC Curve of class {0} (area = {1:0.2f})'.format(classes[i], roc_auc[i]))
        plt.plot(closest_one_fpr[i], closest_one_tpr[i], '.', markersize = 12, fillstyle = 'none', c=color, mew=3)   
        
    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.axes().set_aspect('equal')
    plt.xlim([-0.01, 1.00])
    plt.ylim([-0.01, 1.01])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel('False Positive Rate', fontsize=20)
    plt.ylabel('True Positive Rate', fontsize=20)
    plt.title('Receiver Operating Characteristic Curve', fontsize=24)
    plt.legend(loc="lower right", fontsize=16)
    plt.savefig(f'{pathToFigures}ROC_OVO_Macro_{model_name}.jpg',dpi=150,bbox_inches='tight')
    plt.show()

In [ ]:
roc_curve_multiClassOneVsOneMacro(Y_Pred_Probs,Y_True,numclasses,labels,'vgg16')

In [ ]:
roc_curve_multiClassOneVsOneMacro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_vgg16')

In [ ]:
roc_curve_multiClassOneVsOneMacro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_ResNet50')

In [ ]:
roc_curve_multiClassOneVsOneMacro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_InceptionV3')

In [ ]:
roc_curve_multiClassOneVsOneMacro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_DenseNet121')

In [ ]:
roc_curve_multiClassOneVsOneMacro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_MobileNet')

In [ ]:
roc_curve_multiClassOneVsOneMacro(Y_Pred_Probs,Y_True,numclasses,labels,'HCD-CNN')

In [ ]:
def pr_curve_multiClassOneVsRestMicro(Y_pred_probs,Y_true,num_classes,labels,model_name):
    classes = list(labels.values())
    Y_true = to_categorical(Y_true, num_classes = num_classes,dtype='float64')
    # Compute ROC curve and ROC area for each class
    p = dict()
    r = dict()
    closest_zero = dict()
    closest_zero_p = dict()
    closest_zero_r = dict()
    average_precision = dict()
    pr_auc = dict()
    for i in range(num_classes):
        p[i], r[i], thresholds = precision_recall_curve(Y_true[:, i], Y_pred_probs[:, i])
        average_precision[i] = average_precision_score(Y_true[:, i], Y_pred_probs[:, i])
        closest_zero[i] = np.argmin(np.abs(thresholds))
        closest_zero_p[i] = p[i][closest_zero[i]]
        closest_zero_r[i] = r[i][closest_zero[i]]
    # Compute micro-average ROC curve and ROC area
    p["micro"], r["micro"], thresholds = precision_recall_curve(Y_true.ravel(), Y_pred_probs.ravel())
    average_precision["micro"] = average_precision_score(Y_true, Y_pred_probs, average="micro")
    closest_zero["micro"] = np.argmin(np.abs(thresholds))
    closest_zero_p["micro"] = p["micro"][closest_zero["micro"]]
    closest_zero_r["micro"] = r["micro"][closest_zero["micro"]]
    if num_classes%2==0:
        rows=int(np.ceil(num_classes/2))+1
    else:
        rows=int(np.ceil(num_classes/2))
    cols=2
    plt.tight_layout()
    lw = 2
    plt.figure(figsize=(6*cols,5*rows))
    plt.subplots_adjust(wspace=0.4, hspace=0.4)
    for i in range(num_classes):
        pos=int(str(rows)+str(cols)+str(i+1))
        ax=plt.subplot(pos)
        ax.set_aspect('equal')
        ax.plot(r[i], p[i], color='darkorange', lw=lw, label='PR curve (AP = %0.2f)' % average_precision[i])
        ax.plot(closest_zero_r[i], closest_zero_p[i], 'o', markersize = 12, fillstyle = 'none', c='r', mew=3)
        ax.set_xlim([-0.01, 1.00])
        ax.set_ylim([-0.01, 1.01])
        ax.set_xlabel('Recall', fontsize=14)
        ax.set_ylabel('Precision', fontsize=14)
        ax.set_title(f'Precision Recall Curve for Class:{classes[i]}', fontsize=16)
        ax.legend(loc="lower right", fontsize=14)
        ax.tick_params(labelsize=12)
    pos=int(str(rows)+str(cols)+str(i+2))
    ax=plt.subplot(pos)
    ax.set_aspect('equal')
    ax.plot(r["micro"], p["micro"], color='red', linestyle='--', linewidth=2, label='PR curve Micro-avg. (AP = %0.2f)' % average_precision["micro"])
    ax.plot(closest_zero_r["micro"], closest_zero_p["micro"], 'o', markersize = 12, fillstyle = 'none', c='r', mew=3)
    ax.set_xlim([-0.01, 1.00])
    ax.set_ylim([-0.01, 1.01])
    ax.set_xlabel('Recall', fontsize=14)
    ax.set_ylabel('Precision', fontsize=14)
    ax.set_title('PR Curve: Micro-avg. over all classes', fontsize=16)
    ax.legend(loc="lower right", fontsize=14)
    ax.tick_params(labelsize=12)
    plt.savefig(f'{pathToFigures}PR_OVR_Micro_{model_name}.jpg',dpi=150,bbox_inches='tight')
    plt.show()

In [ ]:
pr_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'vgg16')

In [ ]:
pr_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_vgg16')

In [ ]:
pr_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_ResNet50')

In [ ]:
pr_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_InceptionV3')

In [ ]:
pr_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_DenseNet121')

In [ ]:
pr_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_MobileNet')

In [ ]:
pr_curve_multiClassOneVsRestMicro(Y_Pred_Probs,Y_True,numclasses,labels,'HCD-CNN')

In [ ]:
def pr_curve_multiClassOneVsOne(Y_pred_probs,Y_true,num_classes,labels,model_name):
    classes = list(labels.values())
   # Y_true = label_binarize(Y_true, classes=list(np.arange(num_classes)))
    Y_true = to_categorical(Y_true, num_classes = num_classes,dtype='float64')

    # Compute ROC curve and ROC area for each class
    p = dict()
    r = dict()
    lines = []
    labels = []
    closest_zero = dict()
    closest_zero_p = dict()
    closest_zero_r = dict()
    f_scores = np.linspace(0.2, 0.8, num=4)
    plt.figure(figsize=(10,10))
    plt.subplots_adjust(bottom=0.25)
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
        plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

    lines.append(l)
    labels.append('iso-f1 curves')
    
    average_precision = dict()
    for i in range(num_classes):
        p[i], r[i], thresholds = precision_recall_curve(Y_true[:, i], Y_pred_probs[:, i])
        closest_zero[i] = np.argmin(np.abs(thresholds))
        closest_zero_p[i] = p[i][closest_zero[i]]
        closest_zero_r[i] = r[i][closest_zero[i]]
        average_precision[i] = average_precision_score(Y_true[:, i], Y_pred_probs[:, i])
        

    # Compute micro-average ROC curve and ROC area
    p["micro"], r["micro"], thresholds = precision_recall_curve(Y_true.ravel(), Y_pred_probs.ravel())
    average_precision["micro"] = average_precision_score(Y_true, Y_pred_probs, average="micro")
    closest_zero["micro"] = np.argmin(np.abs(thresholds))
    closest_zero_p["micro"] = p["micro"][closest_zero["micro"]]
    closest_zero_r["micro"] = r["micro"][closest_zero["micro"]]
    # Plot all ROC curves
    
    lw = 2
    l, = plt.plot(r["micro"], p["micro"],lw=lw,color='red', linestyle=':', linewidth=4)
    plt.plot(closest_zero_r["micro"], closest_zero_p["micro"], 'o', markersize = 12, fillstyle = 'none', c='r', mew=3)
    lines.append(l)
    labels.append('micro-average Precision-recall (area = {0:0.2f})'.format(average_precision["micro"]))
    
    colors = cycle(['navy','turquoise','teal', 'darkorange', 
                    'cornflowerblue','brown','purple','green','pink','cyan'])
    for i, color in zip(range(num_classes), colors):
        l, = plt.plot(r[i], p[i], color=color, lw=lw)
        lines.append(l)
        plt.plot(closest_zero_r[i], closest_zero_p[i], 'o', markersize = 12, fillstyle = 'none', c='r', mew=3)   
        labels.append('Precision-recall for class {0} (area = {1:0.2f})'.format(classes[i], average_precision[i]))

    plt.xlim([-0.01, 1.00])
    plt.ylim([-0.01, 1.01])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel('Recall', fontsize=20)
    plt.ylabel('Precision', fontsize=20)
    plt.title('Precision vs. Recall Curve', fontsize=24)
    plt.legend(lines, labels, loc=(0, -.58), fontsize=16)
    plt.axes().set_aspect('equal')
    plt.savefig(f'{pathToFigures}PR_OVO_Micro_{model_name}.jpg',dpi=150,bbox_inches='tight')
    plt.show()

In [ ]:
pr_curve_multiClassOneVsOne(Y_Pred_Probs,Y_True,numclasses,labels,'vgg16')

In [ ]:
pr_curve_multiClassOneVsOne(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_vgg16')

In [ ]:
pr_curve_multiClassOneVsOne(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_ResNet50')

In [ ]:
pr_curve_multiClassOneVsOne(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_InceptionV3')

In [ ]:
pr_curve_multiClassOneVsOne(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_DenseNet121')

In [ ]:
pr_curve_multiClassOneVsOne(Y_Pred_Probs,Y_True,numclasses,labels,'pretrain_MobileNet')

In [ ]:
pr_curve_multiClassOneVsOne(Y_Pred_Probs,Y_True,numclasses,labels,'HCD-CNN')

# Predicting Class of New Test image:

In [ ]:
lesion_type_dict = {
    0: 'No Cancer',
    1: 'Cancer'
}

In [ ]:
def preprocess_input(model_name,X):
    height,width,channels = X.shape
    preprocess_func = None
    def zscoreNorm(x):
        return (x - x.mean()) / x.std()  if x.std() > 0 else x
    def minmax(x):
        return x/255.0
        
    if model_name=='vgg16':
        preprocess_func = minmax
    if model_name=='pretrain_vgg16':
        preprocess_func = keras.applications.vgg16.preprocess_input
    if model_name=='pretrain_ResNet50':
        preprocess_func = keras.applications.resnet.preprocess_input
    if model_name=='pretrain_InceptionV3':
        preprocess_func = keras.applications.inception_v3.preprocess_input
    if model_name=='pretrain_DenseNet121':
        preprocess_func = keras.applications.densenet.preprocess_input
    if model_name=='pretrain_MobileNet':
        preprocess_func = keras.applications.mobilenet.preprocess_input
    if model_name=='HCD-CNN':
        preprocess_func = minmax

    return preprocess_func(img)

In [ ]:
weightsPath='./'
def predictOnImage(img_path,model_name,input_shape):
    ### END CODE HERE ###
    img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
    imshow(img)
    
    model = load_model(weightsPath+model_name+".h5")
    
    x = np.array(img)
    #x = image.img_to_array(img)
    x = preprocess_input(model_name, x)
    imshow(Image.fromarray(x))
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    pred = np.argmax(pred,axis = 1)
    pred = lesion_type_dict[lbls_present[pred[0]]]
    print(f'The image {img_path} is {pred}')

In [ ]:
predictOnImage("your_path_to_image",'HCD-CNN',input_shape)

# SVM

In [ ]:
height=96
width=96
depth=3
input_shape = (height, width, depth)
chanDim = -1

In [ ]:
try:
    os.mkdir('img_dirAll')
except FileExistsError:
    pass
#img_dir=os.path.join('../input/output/','img_dirAll')
img_dir='img_dirAll/'
dfData=shuffle(pd.concat([filenames,labels],axis=1).reset_index(drop=True))
for subf in ['0','1']:
    try:
        os.makedirs(os.path.join(img_dir, subf))
    except FileExistsError:
        pass
src_path='../input/histopathologic-cancer-detection/train'

for i in range(len(dfData)):
    fname=dfData.iloc[i,0] + '.tif'
    label=dfData.iloc[i,1]
    src = os.path.join(src_path, fname)
    dst = os.path.join(img_dir, str(label), fname)
    shutil.copyfile(src, dst)

In [ ]:
testPath='../input/histopathological-cancer-detection-weights/test/test'

def getFeatures(model_name,weightPath,pathToEF,input_shape,path,noOfSamples):
    base_model = load_model(weightPath+model_name+".h5")
    if  model_name=='vgg16' or model_name=='VGG16Pretrained':
        featureLayer='FC2'
    else:
        featureLayer='Act_ReLU_final'
    model = Model(inputs=base_model.input, outputs=base_model.get_layer(featureLayer).output)
    dataGen = getTestGen(model_name,path,input_shape,batch_size=32)
    X_features = model.predict_generator(dataGen,steps=np.ceil(noOfSamples/batch_size), verbose=1)
    Y_True = dataGen.classes
    filenames = dataGen.filenames
    labels = dataGen.class_indices
    labels = dict((v,k) for k,v in labels.items())
    Y_true_ClassNames = [labels[k] for k in Y_True]
        
    cols = list(range(X_features.shape[1]))
    featureDF = pd.DataFrame(data=list(X_features),columns=cols)
    results=pd.DataFrame({"Filename":filenames,
                            "True Class Names":Y_true_ClassNames,
                            "True Class Labels":Y_True})
    results = pd.concat([results,featureDF],axis=1)
    results.to_csv(f"{pathToExtractedFeatures}{model_name}Features.csv",index=False)
    del dataGen    
    del model
    del base_model
    return results

In [ ]:
num_samples

In [ ]:
weightsPath='../input/histopathological-cancer-detection-weights/'
num_samples = len(dfData)
pathToExtractedFeatures = './'
getFeatures('vgg16',weightsPath,pathToExtractedFeatures,input_shape,img_dir,num_samples)

In [ ]:
def get_DatasetFromTrainedModel(model_name,pathToEF):
    dfDataSet = pd.read_csv(f"{pathToEF}{model_name}Features.csv")
    dfDataSet=shuffle(dfDataSet)
    print ("Dataset Length: ", len(dfDataSet)) 
    print ("Dataset Shape: ", dfDataSet.shape) 
    print(dfDataSet.head(10))
    X = dfDataSet.iloc[:,3:-1]
    Y = dfDataSet.iloc[:,2]
    X_trn, X_test,Y_trn, Y_test = train_test_split(X,Y, test_size=0.10, random_state=101, shuffle=True, stratify=Y) 
    X_train,X_val,Y_train,Y_val = train_test_split(X_trn,Y_trn, test_size=0.10, random_state=101, shuffle=True, stratify=Y_trn)
    
    classCountsTrain=pd.DataFrame(np.array(np.unique(Y_train, return_counts=True)).T, columns=['Class','Freq']).set_index('Class')
    classCountsTrain['Freq'] = classCountsTrain['Freq'] / np.sum(classCountsTrain['Freq'])

    #print(classCountsTrain)
    classCountsTrain.plot(kind='bar',label='Class',color=[['royalblue', 'ORANGERED', 'purple', 'GOLD', 'purple','SLATEBLUE','LIMEGREEN']])
    plt.title('Class Proportion in Training:')
    plt.xlabel('dx')
    plt.ylabel('Counts')
    plt.grid(axis='y')

    classCountsVal=pd.DataFrame(np.array(np.unique(Y_val, return_counts=True)).T, columns=['Class','Freq']).set_index('Class')
    classCountsVal['Freq'] = classCountsVal['Freq'] / np.sum(classCountsVal['Freq'])
    #print(classCountsVal)
    classCountsVal.plot(kind='bar', label='Class', color=[['royalblue', 'ORANGERED', 'purple', 'GOLD', 'purple','SLATEBLUE','LIMEGREEN']])
    plt.title('Class Proportion in Validation:')
    plt.xlabel('dx')
    plt.ylabel('Counts')
    plt.grid(axis='y')
    return X_train,Y_train, X_val, Y_val, X_test, Y_test

In [ ]:
def get_DatasetFromTrainedModel(model_name,pathToEF):
    dfDataSet = pd.read_csv(f"{pathToEF}{model_name}Features.csv", header=None)
    dfDataSet=shuffle(dfDataSet)
    print ("Dataset Length: ", len(dfDataSet)) 
    print ("Dataset Shape: ", dfDataSet.shape) 
    print(dfDataSet.head(10))
    X = dfDataSet.iloc[:,3:-1]
    sc_X = StandardScaler()
    X = sc_X.fit_transform(X)
    Y = dfDataSet.iloc[:,2]
    X_train, X_test,Y_train, Y_test = train_test_split(X,Y, test_size=0.10, random_state=101, shuffle=True, stratify=Y) 
    
    classCountsTrain=pd.DataFrame(np.array(np.unique(Y_train, return_counts=True)).T, columns=['Class','Freq']).set_index('Class')
    classCountsTrain['Freq'] = classCountsTrain['Freq'] / np.sum(classCountsTrain['Freq'])

    #print(classCountsTrain)
    classCountsTrain.plot(kind='bar',label='Class',color=[['royalblue', 'ORANGERED', 'purple', 'GOLD', 'purple','SLATEBLUE','LIMEGREEN']])
    plt.title('Class Proportion in Training:')
    plt.xlabel('dx')
    plt.ylabel('Counts')
    plt.grid(axis='y')

    classCountsVal=pd.DataFrame(np.array(np.unique(Y_test, return_counts=True)).T, columns=['Class','Freq']).set_index('Class')
    classCountsVal['Freq'] = classCountsVal['Freq'] / np.sum(classCountsVal['Freq'])
    #print(classCountsVal)
    classCountsVal.plot(kind='bar', label='Class', color=[['royalblue', 'ORANGERED', 'purple', 'GOLD', 'purple','SLATEBLUE','LIMEGREEN']])
    plt.title('Class Proportion in Validation:')
    plt.xlabel('dx')
    plt.ylabel('Counts')
    plt.grid(axis='y')
    return X_train,Y_train, X_test, Y_test

In [ ]:
 X_train,Y_train, X_val, Y_val, X_test, Y_test=get_DatasetFromTrainedModel('vgg16',pathToExtractedFeatures)

## Support Vector Machine Classifier:
<img src='images/svm.jpg' style='height=399px;width=703px'>

### sklearn.svm.SVC:
 - *Support Vector Classification.*
 - The multiclass support is handled according to a one-vs-one scheme.
 - *Parameters:*
    - *C:* float, default=1.0, *Regularization parameter*. The strength of the regularization is inversely proportional to C. Must be strictly positive. The penalty is a squared l2 penalty.
    - *kernel:* {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
    - degree: int, default=3, Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.
    - *gamma:* {‘scale’, ‘auto’} or float, default=’scale’, Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
        - if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,
        - if ‘auto’, uses 1 / n_features.
    - coef0: float, default=0.0, Independent term in kernel function. It is only significant in ‘poly’ and ‘sigmoid’.
    - *degree:* int, default=3, Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.
    - *probability:* bool, default=False, Whether to enable probability estimates. 
      - This must be enabled prior to calling fit.
      - It will slow down that method as it internally uses 5-fold cross-validation.
      - And predict_proba may be inconsistent with predict.
    - *decision_function_shape:* {‘ovo’, ‘ovr’}, default=’ovr’. 
      - Whether to return a one-vs-rest (‘ovr’) decision function of shape (n_samples, n_classes) as all other classifiers.
      - Or the original one-vs-one (‘ovo’) decision function of libsvm which has shape (n_samples, n_classes * (n_classes - 1) / 2). 
      - However, one-vs-one (‘ovo’) is always used as multi-class strategy. The parameter is ignored for binary classification.
    - *class_weight:* dict or ‘balanced’, default=None. 
      - Set the parameter C of class i to class_weight[i]*C for SVC. 
      - If not given, all classes are supposed to have weight one. 
      - The *“balanced”* mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))
 - *Attributes:*
    - *support_:* ndarray of shape (n_SV,), *Indices of support vectors*.
    - *support_vectors_:* ndarray of shape (n_SV, n_features), *Support vectors.*
    - *n_support_:* ndarray of shape (n_class,), dtype=int32, Number of support vectors for each class.
    - *classes_:* ndarray of shape (n_classes,). The classes labels.
 - *Methods:*
   - *fit(X, y, sample_weight=None):* Fit the SVM model according to the given training data.
   - *predict(X):* Perform classification on samples in X. Returns *y_pred* a ndarray of shape *(n_samples,)*, Class labels for samples in X.
   - *score(X, y[, sample_weight]):* Return the mean accuracy on the given test data and labels.
     - *Returns:*
       - *score:* float, Mean accuracy of self.predict(X) wrt. y.
 - *Property:*
   - *predict_proba(X):* Compute probabilities of possible outcomes for samples in X. The model need to have probability information computed at training time: fit with attribute probability set to True.
     - *Returns:*
       - *T:* ndarray of shape (n_samples, n_classes). 
         - Returns the probability of the sample for each class in the model. 
         - The columns correspond to the classes in sorted order, as they appear in the attribute classes_.

In [ ]:
#Simple SVC:
svmClf = SVC(kernel='rbf', gamma='scale', class_weight='balanced', decision_function_shape='ovr')
svmClf.fit(X_train,Y_train)
print('Accuracy Score: ', svmClf.score(X_test, Y_test))
print("#Support Vectors:",sum(svmClf.n_support_))

# Grid Search:

In [ ]:
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100,'scale'], 'C':[0.01, 1, 10]}
svmClf = SVC(kernel='rbf', class_weight='balanced', decision_function_shape='ovr', probability=True)
grid_clf_acc = GridSearchCV(svmClf, param_grid = grid_values)
grid_clf_acc.fit(X_train,Y_train)
print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)
print('Score (ACC): ', grid_clf_acc.score(X_test, Y_test))
print("#Support Vectors:",sum(grid_clf_acc.n_support_))

In [ ]:
grid_clf_auc = GridSearchCV(svmClf, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train,Y_train)
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)
print('Score (AUC): ', grid_clf_auc.score(X_test, Y_test))
print("#Support Vectors:",sum(grid_clf_auc.n_support_))

In [ ]:
X_train,Y_train, X_test, Y_test = get_DatasetFromTrainedModel('vgg16',pathToExtractedFeatures)

In [ ]:
Y_Pred=grid_clf_auc.predict(X_test)
Y_Pred_Probs = grid_clf_auc.predict_proba(X_test)

In [ ]:
model = grid_clf_auc

## Cross-Validation Score:
 - *sklearn.model_selection.cross_val_score*
 - *Parameters:*
   - *estimator:* estimator object implementing ‘fit’. The object to use to fit the data.
   - *X:* array-like of shape (n_samples, n_features) The data to fit. Can be for example a list, or an array.
   - *y:* array-like of shape (n_samples,) or (n_samples, n_outputs), default=None. The target variable to try to predict in the case of supervised learning.
   - *scoring:* str or callable, default=None (Accuracy). 
     - A str or a scorer callable object / function with signature scorer(estimator, X, y) which should return only a single value, to evaluate the predictions on the test set.
     - Similar to cross_validate but only a single metric is permitted.
     - If None, the estimator’s default scorer (if available) is used.
   - *cv:* int, cross-validation generator or an iterable, default=None. Determines the cross-validation splitting strategy. Possible inputs for cv are:
     - *None*: to use the default 5-fold cross validation,
     - *int*: to specify the number of folds in a (Stratified)KFold.
 - *Returns:*
   - *scores:* array of float, shape=(len(list(cv)),) Array of scores of the estimator for each run of the cross validation.

In [ ]:
# accuracy is the default scoring metric
print('Cross-validation (accuracy)', cross_val_score(model, X_test, Y_test, cv=5))
# use AUC as scoring metric
print('Cross-validation (AUC)', cross_val_score(model, X_test, Y_test, cv=5, scoring = 'roc_auc'))
# use precision as scoring metric
print('Cross-validation (precision)', cross_val_score(model, X_test, Y_test, cv=5, scoring = 'precision'))
# use recall as scoring metric
print('Cross-validation (recall)', cross_val_score(model, X_test, Y_test, cv=5, scoring = 'recall'))

In [ ]:
import joblib
def saveModelJL(model_name, model,path):
    filename = f"{path}{model_name}finalized.sav"
    joblib.dump(model, filename)

def loadModelJL(model_name,path):
    filename = f"{path}{model_name}finalized.sav"
    model = joblib.load(filename)
    return model

In [ ]:
saveModelJL(model_name,model,base_dir)

In [ ]:
model = loadModelJL(model_name,base_dir)